<a href="https://colab.research.google.com/github/ale-chen/Arbitrage/blob/main/Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import tensorflow as tf
import matplotlib.pyplot as plt

print("TensorFlow version: {}".format(tf.__version__))

TensorFlow version: 2.11.0


In [2]:
import pandas as pd

X = pd.read_csv('drive/MyDrive/prem_data/train_data/X.csv', header = None)
y = pd.read_csv('drive/MyDrive/prem_data/train_data/y.csv', header = None)

In [3]:
col_names = ['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG',
       'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY',
       'HR', 'AR', 'MAXH', 'MAXD', 'MAXA']
print(col_names)
extended_col_names = col_names.copy()
for i in range(5):
  extended_col_names.extend([name + "_H_" + str(i) for name in col_names])
for i in range(5):
  extended_col_names.extend([name + "_A_" + str(i) for name in col_names])

extended_col_names = [ele for ele in extended_col_names if ele not in ['MAXH','MAXD','MAXA']]

['Date', 'HomeTeam', 'AwayTeam', 'FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG', 'HTR', 'HS', 'AS', 'HST', 'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY', 'HR', 'AR', 'MAXH', 'MAXD', 'MAXA']


In [4]:
X.columns = extended_col_names

#WE DON'T WANT TO TRAIN ON GAME STATS, ONLY PREVIOUS
X.drop(['FTHG', 'FTAG', 'FTR', 'HTHG', 'HTAG','HTR', 'HS', 'AS', 'HST',
        'AST', 'HF', 'AF', 'HC', 'AC', 'HY', 'AY','HR', 'AR'], axis = 1)

,Date,HomeTeam,AwayTeam,Date_H_0,HomeTeam_H_0,AwayTeam_H_0,FTHG_H_0,FTAG_H_0,FTR_H_0,HTHG_H_0,...,AF_A_4,HC_A_4,AC_A_4,HY_A_4,AY_A_4,HR_A_4,AR_A_4,MAXH_A_4,MAXD_A_4,MAXA_A_4
0,733305,5,4,733270,5,10,0.0,1.0,1,0.0,...,16.0,8.0,7.0,2.0,3.0,0.0,0.0,2.38,3.30,3.35
1,733305,6,11,733270,6,18,2.0,1.0,-1,2.0,...,21.0,2.0,6.0,1.0,2.0,0.0,0.0,2.10,3.50,3.80
2,733305,1,0,733270,1,12,3.0,1.0,-1,3.0,...,10.0,7.0,17.0,2.0,2.0,0.0,0.0,6.50,3.80,1.65
3,733305,10,12,733270,5,10,0.0,1.0,1,0.0,...,10.0,20.0,3.0,1.0,1.0,0.0,0.0,1.22,6.50,17.00
4,733306,3,2,733270,3,15,2.0,1.0,-1,1.0,...,12.0,4.0,6.0,0.0,1.0,0.0,0.0,3.30,3.40,2.40
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5404,738555,9,29,733271,9,11,1.0,1.0,0,1.0,...,9.0,6.0,7.0,2.0,2.0,0.0,0.0,3.50,3.40,2.45
5405,738555,20,10,733634,20,6,0.0,2.0,1,0.0,...,10.0,20.0,3.0,1.0,1.0,0.0,0.0,1.22,6.50,17.00
5406,738556,13,17,733270,4,13,2.0,1.0,-1,0.0,...,14.0,8.0,5.0,0.0,1.0,0.0,0.0,2.10,3.35,4.00
5407,738556,39,37,738373,11,39,2.0,0.0,-1,0.0,...,12.0,8.0,3.0,2.0,1.0,0.0,0.0,2.55,3.31,3.07


In [5]:
n_classes = 4

#Y is read in horizontally, so transpose
y = y.transpose()

In [6]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

X_tr, X_te, y_tr, y_te = train_test_split(X, y, test_size = .1, random_state=42)

scaler = StandardScaler()

X_tr_scaled = scaler.fit_transform(X_tr)
X_te_scaled = scaler.transform(X_te)

In [7]:
X_tr_scaled

array([[-0.43852807, -0.10913191, -1.2754195 , ...,  0.48868313,
        -0.48323008, -0.60649115],
       [-0.62792897, -0.50033525, -0.88468068, ..., -0.39271273,
        -0.40167989, -0.34412965],
       [ 0.27041002, -1.08714027,  1.55743693, ...,  2.28819968,
         0.19363646, -0.74329393],
       ...,
       [ 1.62317964, -0.8915386 ,  1.85049104, ..., -0.31926308,
        -0.45061001, -0.34412965],
       [ 1.72379887,  1.26007979, -0.68931127, ...,  0.48868313,
        -0.52400517, -0.60649115],
       [-1.18955525, -0.10913191, -0.39625716, ..., -0.39271273,
        -0.40167989, -0.34412965]])

In [18]:
from tensorflow import keras

def preprocess(x, y):
  return x, y

def create_dataset(xs, ys, n_classes=4):
  ys = tf.one_hot(ys, depth=n_classes)
  return tf.data.Dataset.from_tensor_slices((xs, ys)) \
    .map(preprocess) \
    .shuffle(len(ys)) \
    .batch(128)

In [19]:
train_set = create_dataset(X_tr_scaled, y_tr)
test_set = create_dataset(X_te_scaled, y_te)

In [15]:
print(y_tr.shape)

(4868, 1)


In [10]:
X_tr_scaled.shape

(4868, 261)

In [11]:
model = keras.Sequential([
    keras.layers.Dense(units=261, activation='relu', input_shape = (261,)),
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dense(units=4, activation='softmax')
])

In [12]:

model.compile(optimizer='adam', 
              loss=tf.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(
    train_set.repeat(), 
    epochs=10, 
    steps_per_epoch=500,
    validation_data=test_set.repeat(), 
    validation_steps=2
)

Epoch 1/10


ValueError: ignored

In [ ]:
predictions = model(X_tr)
predictions[:5]

AttributeError: ignored